In [1]:
# Import Packages
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

In [2]:
# Load dataset
mnist_dataset, mnist_info = tfds.load('mnist', with_info= True, as_supervised= True)

mnist_info

tfds.core.DatasetInfo(
    name='mnist',
    version=3.0.1,
    description='The MNIST database of handwritten digits.',
    homepage='http://yann.lecun.com/exdb/mnist/',
    features=FeaturesDict({
        'image': Image(shape=(28, 28, 1), dtype=tf.uint8),
        'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=10),
    }),
    total_num_examples=70000,
    splits={
        'test': 10000,
        'train': 60000,
    },
    supervised_keys=('image', 'label'),
    citation="""@article{lecun2010mnist,
      title={MNIST handwritten digit database},
      author={LeCun, Yann and Cortes, Corinna and Burges, CJ},
      journal={ATT Labs [Online]. Available: http://yann. lecun. com/exdb/mnist},
      volume={2},
      year={2010}
    }""",
    redistribution_info=,
)

In [3]:
# Extract traning set and test set 
mnist_train = mnist_dataset['train']
mnist_test = mnist_dataset['test']

In [6]:
# Split training set into training set and validation set
# Validation set is used to validate model and prevent overfitting
num_validation_samples = 0.1 * mnist_info.splits['train'].num_examples

# Cast the sample numbers into an int number
num_validation_samples = tf.cast(num_validation_samples, tf.int64)

# Get the number of test samples
num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples, tf.int64)

# Data Processing

In [8]:
# Processing data
# Scale
def scale (image, label):
    image = tf.cast(image, tf.float32)
    image = image/255.

    return image, label
# the method .map() allows us to apply a custom transformation to a given dataset
scaled_training_and_validation_data = mnist_train.map(scale)
test_data = mnist_test.map(scale)

# Shuffle data
BUFFER_SIZE = 10000
shuffled_train_and_validation_data = scaled_training_and_validation_data.shuffle(BUFFER_SIZE)

In [9]:
# Get validation dataset
validation_data = shuffled_train_and_validation_data.take(num_validation_samples)

# Get training dataset
train_data = shuffled_train_and_validation_data.skip(num_validation_samples)

In [10]:
# Determine train and validation batches
BATCH_SIZE = 100

train_data = train_data.batch(BATCH_SIZE)

validation_data = validation_data.batch(num_validation_samples)

test_data = test_data.batch(num_test_samples)

validation_inputs, validation_targets = next(iter(validation_data))

# Create Model

## Outline the model: 
* input layer with layers.Flatten
* 2 hidden layers with layers.Dense
* out layer with layers.Dense


In [13]:
input_size = 784 # 28*28 after flatten
output_size = 10

# set hindden layer size
hidden_layer_size = 100

# output layer is to categories, so softmax is used as activiation funtion
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape = (28,28,1)),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(output_size, activation='softmax')
])

## Set objective function and optimization function by compile model
* model.compile(optimizer=, loss=)
* loss function for classification: cross entory
* optimization function: 'adam'

In [17]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

## Train the model
* set epochs
* fit with training data, validation_data

In [20]:
# NUM_EPOCHS = 50
NUM_EPOCHS = 20

model.fit(train_data, epochs=NUM_EPOCHS, validation_data=(validation_inputs,validation_targets), verbose=2)

Epoch 1/20
540/540 - 4s - loss: 0.0043 - accuracy: 0.9984 - val_loss: 0.0060 - val_accuracy: 0.9982
Epoch 2/20
540/540 - 4s - loss: 0.0043 - accuracy: 0.9986 - val_loss: 0.0052 - val_accuracy: 0.9982
Epoch 3/20
540/540 - 3s - loss: 0.0030 - accuracy: 0.9992 - val_loss: 0.0018 - val_accuracy: 0.9992
Epoch 4/20
540/540 - 4s - loss: 0.0027 - accuracy: 0.9990 - val_loss: 0.0014 - val_accuracy: 0.9993
Epoch 5/20
540/540 - 4s - loss: 0.0069 - accuracy: 0.9979 - val_loss: 0.0071 - val_accuracy: 0.9973
Epoch 6/20
540/540 - 4s - loss: 0.0044 - accuracy: 0.9984 - val_loss: 0.0041 - val_accuracy: 0.9982
Epoch 7/20
540/540 - 4s - loss: 0.0053 - accuracy: 0.9984 - val_loss: 0.0164 - val_accuracy: 0.9948
Epoch 8/20
540/540 - 4s - loss: 0.0031 - accuracy: 0.9990 - val_loss: 0.0013 - val_accuracy: 0.9998
Epoch 9/20
540/540 - 4s - loss: 0.0024 - accuracy: 0.9992 - val_loss: 4.7447e-04 - val_accuracy: 1.0000
Epoch 10/20
540/540 - 4s - loss: 0.0069 - accuracy: 0.9980 - val_loss: 0.0096 - val_accuracy: 0.

After 21 epochs, the validation_loss starts to increase, indicating the model is overfitted. 

## Test the model with model.evalute()

In [21]:
test_loss, test_accuracy = model.evaluate(test_data)

1/1 [==============================] - 0s 2ms/step - loss: 0.1746 - accuracy: 0.9785


In [22]:
print(test_loss)
print(test_accuracy)

0.17459875345230103
0.9785000085830688
